In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import mnist  # 导入PyTorch中内置的mnist数据集

In [2]:
# 数据标准化，直接采用参考文档代码
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5      # 标准化，这个技巧之后会讲到
    # x = x.reshape((-1,))   # 拉平
    x = torch.from_numpy(x)  # 将numpy数组转换为tensor
    x = x.unsqueeze(0)       # 增加channel，设置为1。图片的大小为：(1, 28, 28)
    return x
# 重新加载数据集，申明定义的数据变换
train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True)
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

In [3]:
img, label = train_set[0]
img.shape

torch.Size([1, 28, 28])

In [4]:
from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [5]:
image, label = next(iter(train_data))

In [6]:
image.shape

torch.Size([64, 1, 28, 28])

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一层：数据输入是：(batch, 1, 28, 28)，经过卷积，第一层的输出则是：(batch, 25, 26, 26)
        self.layer1 = nn.Sequential(
                      nn.Conv2d(1, 25, 3),
                      nn.ReLU())
        # 第二层是池化层，池化窗口大小为(2, 2)，数据输入是：(batch, 25, 26, 26)，经过池化，第二层的输出则是：(batch, 25, 13, 13)
        # 第三层：数据输入是：(batch, 25, 13, 13)，经过卷积，第三层的输出则是：(batch, 50, 11, 11)
        self.layer3 = nn.Sequential(
                      nn.Conv2d(25, 50, 3),
                      nn.ReLU())
        # 第四层是池化层，池化窗口大小为(2, 2)，数据输入是：(batch, 50, 11, 11)，经过池化，第四层的输出则是：(batch, 50, 5, 5)
        # 最后是线性层，共两层：
        self.fc = nn.Sequential(
                      nn.Linear(50 * 5 * 5, 1024),
                      nn.ReLU(),
                      nn.Linear(1024, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10))
        self.pool = nn.MaxPool2d(2, 2)
    def forward(self, x):
        out = self.layer1(x)
        out = self.pool(out)
        out = self.layer3(out)
        out = self.pool(out)
        out = out.view(out.shape[0], -1)  # 将图片拉伸（展平），变成向量，向量的大小为 50 X 5 X 5，out.shape[0]等于batch
        out = self.fc(out)
        return out

In [8]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [9]:
# 训练模型
Epochs = 5
for epoch in range(Epochs):
    loss_sum = 0
    acc_sum = 0
    for image, label in train_data:
        image = Variable(image)
        label = Variable(label)
        # 前向传播
        y_pred = net(image)
        loss = criterion(y_pred, label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_sum += loss.data.item()
        _, out = y_pred.max(1)                     # 返回每一行最大值对应的下标，就是图片的预测值
        num_correct = (out == label).sum().item()  # 统计预测正确的数量
        acc = num_correct / image.shape[0]         # 得到这一个batch的平均准确率
        acc_sum += acc
    ave_train_loss = loss_sum / len(train_data)
    ave_train_acc = acc_sum / len(train_data)
    loss_sum = 0
    acc_sum = 0
    # 在测试集上检验效果
    net.eval()  # 将模型改为预测模式，eval（）时，pytorch会自动把BN和DropOut固定住，不会取平均，而是用训练好的值。
    for image, label in test_data:
        image = Variable(image)
        label = Variable(label)
        y_pred = net(image)
        loss = criterion(y_pred, label)
        loss_sum += loss.data.item()
        _, out = y_pred.max(1)
        num_correct = (out == label).sum().item()
        acc = num_correct / image.shape[0]
        acc_sum += acc
    ave_test_loss = loss_sum / len(test_data)
    ave_test_acc = acc_sum / len(test_data)
    print('epoch: {:2d}, train loss: {:.4f}, train acc: {:.4f}, test loss: {:.4f}, test acc: {:.4f}'.format(epoch + 1, ave_train_loss, ave_train_acc, ave_test_loss, ave_test_acc))

epoch:  1, train loss: 0.2846, train acc: 0.9095, test loss: 0.0549, test acc: 0.9816
epoch:  2, train loss: 0.0540, train acc: 0.9831, test loss: 0.0650, test acc: 0.9778
epoch:  3, train loss: 0.0356, train acc: 0.9888, test loss: 0.0324, test acc: 0.9880
epoch:  4, train loss: 0.0260, train acc: 0.9920, test loss: 0.0319, test acc: 0.9895
epoch:  5, train loss: 0.0197, train acc: 0.9938, test loss: 0.0281, test acc: 0.9906
